# Financial Data Analysis on Bitso

The estimated information shows the monthly aggregated daily aggregate trading volume of ARS and crypto-assets (BTC and USDT) on bitso. 

This Python code performs various operations for financial data analysis related to Bitso and the cryptocurrency market. Key functions include:

1. Retrieving Bitso ticket data and storing it in Amazon S3.
2. Creating monthly cumulative volume charts.
3. Integrating price data in dollars for additional analysis.
4. Monthly comparison of cumulative volume.

The code utilizes libraries such as pandas, requests, plotly, boto3, and coinmetrics-api-client to carry out these tasks. Additionally, it includes data interpolation and DataFrame manipulation for effective analysis and visualizations.

In [1]:
###########################################
## Request all cryptos and currencies pair
##########################################

import requests
import pandas as pd
import boto3
import io
import os
from datetime import datetime
from kaggle_secrets import UserSecretsClient

# Configuración de credenciales de AWS
user_secrets = UserSecretsClient()
access_key = user_secrets.get_secret("AWSAccessKeyId")
secret_key = user_secrets.get_secret("AWSSecretKey")
bucket_name = 'bitso.api'

# Crear el cliente de S3
s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)

# Lista de prefijos y sufijos
prefijos = ["usd", "usdt", "btc", "eth", "dai"]
sufijos = ["mxn", "ars", "cop", "brl"]

# Función para generar la URL de la API
def generar_url(prefijo, sufijo):
    return f"https://api.bitso.com/v3/ticker?book={prefijo}_{sufijo}"

# Función para obtener datos de un ticket y guardar en el bucket con subcarpetas
def obtener_y_guardar(prefijo, sufijo):
    # Generar la URL de la API
    url = generar_url(prefijo, sufijo)

    # Imprimir la URL generada
    print(f"URL generada: {url}")

    # Realizar la solicitud GET a la API de tickets de Bitso para el ticket específico
    response = requests.get(url)
    data = response.json()

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Verificar si 'payload' está presente en el diccionario de datos
        if 'payload' in data:
            # Crear un DataFrame con los datos del ticket
            df = pd.DataFrame([data['payload']])

            # Nombre del archivo de tickets con la fecha actual
            current_date = datetime.now().strftime("%Y-%m-%d")
            filename = f'datos_{prefijo}_{sufijo}_{current_date}.csv'

            # Ruta completa del objeto en el bucket que incluye las subcarpetas
            s3_object_key = os.path.join(sufijo, prefijo, filename)

            # Guardar el DataFrame en un archivo CSV temporal
            df.to_csv(filename, index=False)

            # Subir el archivo CSV de tickets a S3
            s3_client.upload_file(filename, bucket_name, s3_object_key)

            # Eliminar el archivo temporal local
            os.remove(filename)
        else:
            print("La clave 'payload' no está presente en el diccionario de datos.")
    elif response.status_code == 400 and "Unknown OrderBook" in data.get('error', {}).get('message', ''):
        print(f"Advertencia: La combinación {prefijo}_{sufijo} no está disponible en la API. Saltando esta solicitud.")
    else:
        print(f"Error al realizar la solicitud. Código de estado: {response.status_code}")
        print(f"Contenido de la respuesta: {response.content}")

# Ejemplo de uso para varios prefijos y sufijos
for prefijo in prefijos:
    for sufijo in sufijos:
        obtener_y_guardar(prefijo, sufijo)

URL generada: https://api.bitso.com/v3/ticker?book=usd_mxn
URL generada: https://api.bitso.com/v3/ticker?book=usd_ars
URL generada: https://api.bitso.com/v3/ticker?book=usd_cop
URL generada: https://api.bitso.com/v3/ticker?book=usd_brl
URL generada: https://api.bitso.com/v3/ticker?book=usdt_mxn
URL generada: https://api.bitso.com/v3/ticker?book=usdt_ars
URL generada: https://api.bitso.com/v3/ticker?book=usdt_cop
URL generada: https://api.bitso.com/v3/ticker?book=usdt_brl
URL generada: https://api.bitso.com/v3/ticker?book=btc_mxn
URL generada: https://api.bitso.com/v3/ticker?book=btc_ars
URL generada: https://api.bitso.com/v3/ticker?book=btc_cop
URL generada: https://api.bitso.com/v3/ticker?book=btc_brl
URL generada: https://api.bitso.com/v3/ticker?book=eth_mxn
URL generada: https://api.bitso.com/v3/ticker?book=eth_ars
URL generada: https://api.bitso.com/v3/ticker?book=eth_cop
URL generada: https://api.bitso.com/v3/ticker?book=eth_brl
URL generada: https://api.bitso.com/v3/ticker?book=d

In [2]:
import boto3
import pandas as pd
from io import BytesIO

# Crear el cliente de S3
s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)

# Lista de prefijos y sufijos
prefijos = ["usd", "usdt", "btc", "eth", "dai"]
sufijos = ["mxn", "ars", "cop", "brl"]

# Inicializar una lista para almacenar los DataFrames
dfs = []

# Iterar a través de los prefijos y sufijos
for prefijo in prefijos:
    for sufijo in sufijos:
        # Construir el prefijo y sufijo para formar la carpeta en S3
        carpeta_s3 = f'{sufijo}/{prefijo}/'
        
        # Lista de archivos en S3 para la combinación actual de prefijo y sufijo
        objects = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=carpeta_s3)
        
        # Iterar a través de los objetos en S3
        for obj in objects.get('Contents', []):
            # Obtener el nombre del objeto
            s3_object_key = obj['Key']
            
            # Descargar el archivo CSV desde S3
            s3_response = s3_client.get_object(Bucket=bucket_name, Key=s3_object_key)
            csv_data = s3_response['Body'].read()
            
            # Verificar si el archivo CSV contiene datos
            if len(csv_data) > 0:
                # Crear un DataFrame de pandas desde los datos del CSV
                df = pd.read_csv(BytesIO(csv_data))
                
                # Agregar el DataFrame a la lista
                dfs.append(df)
            else:
                print(f"Advertencia: El archivo {s3_object_key} está vacío. Saltando esta lectura.")

# Combinar todos los DataFrames en uno solo
final_df = pd.concat(dfs, ignore_index=True)

Advertencia: El archivo ars/btc/ está vacío. Saltando esta lectura.


# USD Volume

In [3]:
pip install coinmetrics-api-client -U

  Obtaining dependency information for coinmetrics-api-client from https://files.pythonhosted.org/packages/0a/6a/8d17f70ddfb8aaf0637afaa5a5d2985c405dcbff6417a72f5f7c1410e20c/coinmetrics_api_client-2024.7.11.18-py3-none-any.whl.metadata
  Obtaining dependency information for typer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/0d/44/56c3f48d2bb83d76f5c970aef8e2c3ebd6a832f09e3621c5395371fe6999/typer-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from coinmetrics.api_client import CoinMetricsClient

# Inicializa el cliente de Coin Metrics para la API Comunitaria
client = CoinMetricsClient()

# Define los activos y métricas
assets = ['btc', 'eth', 'usdc', 'usdt', 'dai']
metrics = ['PriceUSD']

# Define la fecha de inicio (January 1, 2023)
start_time = '2023-01-01'

# Obtén los datos de precio de BTC y ETH con frecuencia diaria desde January 1, 2023
price_data = client.get_asset_metrics(assets=assets, metrics=metrics, frequency='1d', start_time=start_time)

# Convierte los datos a un DataFrame de Pandas
price_df = price_data.to_dataframe()

# Asegúrate de que la columna 'PriceUSD' sea numérica
price_df['PriceUSD'] = pd.to_numeric(price_df['PriceUSD'], errors='coerce')

In [5]:
# Asegurémonos de que los campos de fecha tengan el mismo formato y solo consideren la parte del día
final_df['created_at'] = pd.to_datetime(final_df['created_at']).dt.floor('D')
price_df['time'] = pd.to_datetime(price_df['time']).dt.floor('D')

# Extraer los caracteres antes del guion bajo "_" en el campo 'book' en 'final_df'
final_df['book_prefix'] = final_df['book'].str.split('_').str[0]

# Realizar la transformación: cambiar 'usd' a 'usdc'
final_df['book_prefix'] = final_df['book_prefix'].replace('usd', 'usdc')

# Fusionar los DataFrames en función de 'created_at', 'book_prefix', y 'asset'
merged_df = pd.merge(final_df, price_df, left_on=['created_at', 'book_prefix'], right_on=['time', 'asset'], how='inner')

# Realizar la multiplicación del campo 'volume' por 'PriceUSD'
merged_df['volumeUSD'] = merged_df['volume'] * merged_df['PriceUSD']

merged_df['countries'] = merged_df['book'].str[-3:]

In [6]:
merged_df['volumeUSD'] = pd.to_numeric(merged_df['volumeUSD'], errors='coerce')

## Evolución por país y criptoactivo

Por país

In [7]:
import plotly.graph_objects as go
from datetime import datetime
import pandas as pd

# Obtén la fecha de hoy
hoy = datetime.now().date()

# Define la fecha desde la que deseas mostrar los datos (15 de febrero)
fecha_inicio = datetime(2024, 2, 18).date()

# Define la fecha final para mostrar los datos (17 de marzo)
fecha_final = datetime(2024, 7, 29).date()

# Filtra las filas para incluir solo las desde la fecha de inicio hasta la fecha final
total_countries_selected_columns = merged_df[(merged_df['created_at'].dt.date >= fecha_inicio) & (merged_df['created_at'].dt.date <= fecha_final)]

# Selecciona solo las tres columnas sin la de fecha
total_countries_selected_columns = total_countries_selected_columns[['created_at', 'countries', 'volumeUSD']]

# Group by day and country, sum the volumeUSD
total_countries_grouped_df = total_countries_selected_columns.groupby([pd.Grouper(key='created_at', freq='w'), 'countries']).sum().reset_index()

# Create a bar chart with plotly using daily values
total_countries_fig = go.Figure()

# Definir colores para cada país
colores = {
    'mxn': '#CE1126',  # Rojo para México
    'ars': '#75aadb',  # Celeste para Argentina
    'cop': '#FFCD00',  # Amarillo para Colombia
    'brl': '#009739'   # Verde para Brasil
}

for country in total_countries_grouped_df['countries'].unique():
    df_country = total_countries_grouped_df[total_countries_grouped_df['countries'] == country]
    
    total_countries_fig.add_trace(go.Bar(
        x=df_country['created_at'],
        y=df_country['volumeUSD'],
        name=country.upper(),  # Mostrar el nombre del país en mayúsculas
        marker_color=colores[country]  # Añadir el color del país correspondiente
    ))

# Customize the layout
total_countries_fig.update_layout(
    title='<b>Cryptoasset exchange vis-a-vis local currencies by countries<b>',
    title_font_color="#AA322F",    
    xaxis_title='Day',
    yaxis_title='Volume (USD)',
    barmode='stack',  # Ensure bars are stacked
    legend_title='Country',
)

# Show the plot
total_countries_fig.show()


In [8]:
import pandas as pd

# Convertir la columna 'created_at' a tipo datetime
total_countries_grouped_df['created_at'] = pd.to_datetime(total_countries_grouped_df['created_at'])

# Ordenar el DataFrame por fecha ascendente
total_countries_grouped_df = total_countries_grouped_df.sort_values(by='created_at')

# Calcular las variaciones porcentuales entre fechas para cada país
variations_between_dates = total_countries_grouped_df.pivot(index='created_at', columns='countries', values='volumeUSD').pct_change() * 100

# Filtrar las variaciones porcentuales para eliminar NaT
variations_between_dates = variations_between_dates.dropna()

# Calcular las variaciones de cada país para la última fecha respecto de la inicial
initial_values = total_countries_grouped_df.groupby('countries')['volumeUSD'].first()
final_values = total_countries_grouped_df.groupby('countries')['volumeUSD'].last()
structure_percent_change = ((final_values - initial_values) / initial_values) * 100

# Calcular la estructura relativa de países para el primer y último momento
initial_structure = (initial_values / initial_values.sum()) * 100
final_structure = (final_values / final_values.sum()) * 100

# Calcular el cambio en la estructura relativa en puntos porcentuales
structure_change = final_structure - initial_structure

# Calcular la desviación estándar de las variaciones
desv_std_variaciones = variations_between_dates.std()

# Mostrar los resultados en un análisis de texto
print("\nVariaciones Porcentuales Entre Fechas:")
for date, row in variations_between_dates.iterrows():
    print(f"Fecha: {date.strftime('%Y-%m-%d')}")
    for country, variation in row.items():
        if abs(variation) > 0.1 * desv_std_variaciones[country]:
            print(f"- {country}: {variation:.2f}%")

# Mostrar el cambio porcentual en la estructura relativa de países
print("\nCambio porcentual entre el último dato y el inicial:")
for country, percent_change in structure_percent_change.items():
    print(f"- {country}: {percent_change:.2f}%")

print("\nEstructura Relativa de Países:")
print("En el primer momento:")
for country, structure in initial_structure.items():
    print(f"- {country}: {structure:.2f}%")
print("\nEn el último momento:")
for country, structure in final_structure.items():
    print(f"- {country}: {structure:.2f}%")
print("\nCambio en la Estructura Relativa de Países:")
for country, change in structure_change.items():
    print(f"- {country}: {change:.2f} p.p.")



Variaciones Porcentuales Entre Fechas:
Fecha: 2024-03-03
- ars: 81.56%
- brl: 10.64%
- cop: -29.30%
- mxn: 35.39%
Fecha: 2024-03-10
- ars: 22.29%
- brl: 19.52%
- cop: 10.32%
- mxn: 10.72%
Fecha: 2024-03-17
- ars: 4.16%
- brl: 5.03%
- cop: -11.00%
- mxn: -3.08%
Fecha: 2024-03-24
- ars: 9.89%
- brl: -41.78%
- mxn: -23.17%
Fecha: 2024-03-31
- brl: -16.50%
- cop: -47.96%
- mxn: -14.47%
Fecha: 2024-04-07
- ars: -12.71%
- brl: 31.74%
- cop: 28.23%
- mxn: 13.10%
Fecha: 2024-04-14
- ars: 33.76%
- brl: 58.22%
- cop: 42.59%
- mxn: 5.92%
Fecha: 2024-04-21
- ars: -45.21%
- brl: -17.89%
- cop: 8.35%
Fecha: 2024-04-28
- brl: -29.89%
- cop: 8.82%
- mxn: -21.11%
Fecha: 2024-05-05
- ars: 86.20%
- brl: 43.48%
- cop: -26.22%
- mxn: 22.91%
Fecha: 2024-05-12
- brl: -8.90%
- cop: 36.10%
- mxn: -18.53%
Fecha: 2024-05-19
- ars: -41.71%
- brl: -21.33%
- cop: -46.59%
- mxn: 7.71%
Fecha: 2024-05-26
- ars: -9.75%
- brl: 44.42%
- cop: -43.63%
- mxn: 13.86%
Fecha: 2024-06-02
- ars: 37.09%
- brl: -40.82%
- cop: 21.

In [9]:
import numpy as np

# Inicializar el análisis de texto
analysis_text = ""

# Iterar sobre cada país
for country in total_countries_grouped_df['countries'].unique():
    country_data = total_countries_grouped_df[total_countries_grouped_df['countries'] == country]
    analysis_text += f"Análisis de la evolución del intercambio en {country}:\n"
    
    # Calcular las variaciones porcentuales con respecto al primer dato
    first_volume = country_data.iloc[0]['volumeUSD']
    variations = [(volume - first_volume) / first_volume * 100 for volume in country_data['volumeUSD']]
    
    # Definir threshold_variation como dos veces la desviación estándar de las variaciones
    threshold_variation = 2 * np.std(variations)
    
    # Mostrar el threshold_variation estimado
    analysis_text += f"Umbral estimado de variación porcentual relevante: {threshold_variation:.2f}%\n"
    
    # Identificar las variaciones porcentuales que superan el umbral
    for variation_pct, (_, row) in zip(variations, country_data.iterrows()):
        if row.name != 0:  # Comprobamos que no sea el primer dato
            if abs(variation_pct) > threshold_variation:
                analysis_text += f"Variación porcentual relevante del {row['created_at'].strftime('%Y-%m-%d')}: {variation_pct:.2f}%\n"
    
    analysis_text += "\n"

print(analysis_text)


Análisis de la evolución del intercambio en ars:
Umbral estimado de variación porcentual relevante: 128.34%
Variación porcentual relevante del 2024-03-17: 131.27%
Variación porcentual relevante del 2024-03-24: 154.14%
Variación porcentual relevante del 2024-03-31: 151.13%
Variación porcentual relevante del 2024-04-14: 193.21%
Variación porcentual relevante del 2024-05-05: 194.41%
Variación porcentual relevante del 2024-05-12: 186.57%
Variación porcentual relevante del 2024-06-09: 257.18%
Variación porcentual relevante del 2024-07-21: 137.73%

Análisis de la evolución del intercambio en brl:
Umbral estimado de variación porcentual relevante: 44.43%
Variación porcentual relevante del 2024-07-07: 56.01%

Análisis de la evolución del intercambio en cop:
Umbral estimado de variación porcentual relevante: 80.81%

Análisis de la evolución del intercambio en mxn:
Umbral estimado de variación porcentual relevante: 32.33%
Variación porcentual relevante del 2024-03-03: 35.39%
Variación porcentual

In [10]:
###################################
# Cambios semanales más relevantes
##################################

import random
import numpy as np

# Sinónimos para "registraron"
registraron_synonyms = ["registraron", "experimentaron", "mostraron", "reportaron"]

# Sinónimos para "Se observó"
observado_synonyms = ["Se observó", "Se notó", "Se percibió", "Se registró"]

# Sinónimos para "incremento"
incremento_synonyms = ["incremento", "aumento", "crecimiento"]

# Sinónimos para "disminución"
disminucion_synonyms = ["disminución", "reducción", "descenso"]

# Palabras para indicar la dirección de la variación
direction_words = {
    "incremento": ["incrementó", "aumentó", "creció"],
    "disminución": ["disminuyó", "redujo", "descendió"]
}

# Inicializar el análisis de texto
analysis_text = ""

# Inicializar listas para las oraciones de incremento y disminución
incremento_sentences = []
disminucion_sentences = []

# Inicializar un diccionario para almacenar las oraciones de incremento por semana
incremento_by_week = {}
disminucion_by_week = {}

# Iterar sobre cada país
for country in total_countries_grouped_df['countries'].unique():
    country_data = total_countries_grouped_df[total_countries_grouped_df['countries'] == country]
    
    # Calcular las variaciones porcentuales con respecto al dato anterior
    variations = country_data['volumeUSD'].pct_change() * 100
    
    # Eliminar el primer valor de las variaciones (NaN)
    variations = variations.iloc[1:]
    
    # Definir threshold_variation como dos veces la desviación estándar de las variaciones
    threshold_variation = 1 * np.std(variations)
    
    # Identificar las variaciones porcentuales que superan el umbral
    for variation in variations:
        if abs(variation) > threshold_variation:
            date_index = variations.index[variations == variation][0]
            date = country_data.loc[date_index, 'created_at'].strftime('%d de %B')
            # Selección aleatoria de sinónimos
            observado_synonym = random.choice(observado_synonyms)
            registraron_synonym = random.choice(registraron_synonyms)
            direction_word = random.choice(direction_words["incremento"]) if variation > 0 else random.choice(direction_words["disminución"])
            if variation > 0:
                incremento_synonym = random.choice(incremento_synonyms)
                incremento_sentences.append(f"{observado_synonym} en la semana de {date}, los datos de {country} {registraron_synonym} un {incremento_synonym} del {abs(variation):.2f}%, lo que {direction_word} con respecto a la semana anterior.")
                # Almacenar la oración de incremento en el diccionario por semana
                if date not in incremento_by_week:
                    incremento_by_week[date] = []
                incremento_by_week[date].append(f"{country}: {abs(variation):.2f}%")
            else:
                disminucion_synonym = random.choice(disminucion_synonyms)
                disminucion_sentences.append(f"{observado_synonym} en la semana de {date}, los datos de {country} {registraron_synonym} una {disminucion_synonym} del {abs(variation):.2f}%, lo que {direction_word} con respecto a la semana anterior.")
                # Almacenar la oración de disminución en el diccionario por semana
                if date not in disminucion_by_week:
                    disminucion_by_week[date] = []
                disminucion_by_week[date].append(f"{country}: {abs(variation):.2f}%")

# Generar el análisis final
for week in sorted(set(incremento_by_week.keys()) | set(disminucion_by_week.keys())):
    incremento_sentences_week = incremento_by_week.get(week, [])
    disminucion_sentences_week = disminucion_by_week.get(week, [])
    
    if incremento_sentences_week and disminucion_sentences_week:
        incremento_sentence = " ".join(incremento_sentences_week)
        disminucion_sentence = " ".join(disminucion_sentences_week)
        analysis_text += f"En la semana de {week}, {incremento_sentence} Sin embargo, {disminucion_sentence}\n\n"
    elif incremento_sentences_week:
        incremento_sentence = " ".join(incremento_sentences_week)
        analysis_text += f"En la semana de {week}, {incremento_sentence}\n\n"
    elif disminucion_sentences_week:
        disminucion_sentence = " ".join(disminucion_sentences_week)
        analysis_text += f"En la semana de {week}, {disminucion_sentence}\n\n"

# Agregar los umbrales estimados al final
analysis_text += "Umbrales estimados de variación porcentual relevante:\n"
for country in total_countries_grouped_df['countries'].unique():
    country_variations = total_countries_grouped_df[total_countries_grouped_df['countries'] == country]['volumeUSD'].pct_change() * 100
    country_variations = country_variations.iloc[1:]
    threshold_variation = 1 * np.std(country_variations)
    analysis_text += f"{country}: {threshold_variation:.2f}%\n"

print(analysis_text)


En la semana de 02 de June, brl: 40.82% mxn: 22.13%

En la semana de 03 de March, ars: 81.56% mxn: 35.39%

En la semana de 05 de May, ars: 86.20% brl: 43.48% mxn: 22.91%

En la semana de 07 de April, brl: 31.74%

En la semana de 07 de July, brl: 53.34%

En la semana de 09 de June, ars: 72.84% brl: 47.15% cop: 227.15% mxn: 36.60%

En la semana de 12 de May, mxn: 18.53%

En la semana de 14 de April, brl: 58.22%

En la semana de 14 de July, brl: 39.28%

En la semana de 16 de June, ars: 57.04%

En la semana de 19 de May, ars: 41.71%

En la semana de 21 de April, ars: 45.21%

En la semana de 21 de July, ars: 41.35%

En la semana de 24 de March, brl: 41.78% mxn: 23.17%

En la semana de 26 de May, brl: 44.42%

En la semana de 28 de April, mxn: 21.11%

En la semana de 30 de June, ars: 50.49%

Umbrales estimados de variación porcentual relevante:
ars: 38.94%
brl: 31.40%
cop: 53.53%
mxn: 16.66%



In [11]:
###########################
# Respecto al dato inicial
###########################

# Inicializar el análisis de texto
analysis_text = ""

# Inicializar un diccionario para almacenar las variaciones por país
variations = {}

# Iterar sobre cada país
for country in total_countries_grouped_df['countries'].unique():
    country_data = total_countries_grouped_df[total_countries_grouped_df['countries'] == country]
    
    # Verificar si hay suficientes datos disponibles para el país
    if len(country_data) > 1:
        # Obtener el valor inicial y el último valor disponible
        initial_value = country_data.iloc[0]['volumeUSD']
        last_value = country_data.iloc[-1]['volumeUSD']
        
        # Calcular la variación porcentual entre el último valor disponible y el valor inicial
        variation_pct = (last_value - initial_value) / initial_value * 100
        
        # Almacenar la variación en el diccionario
        if variation_pct > 0:
            direction = 'aumentó'
        elif variation_pct < 0:
            direction = 'disminuyó'
        else:
            direction = 'se mantuvo igual'
        
        if direction not in variations:
            variations[direction] = []
        variations[direction].append((country, variation_pct))
    else:
        analysis_text += f"No hay suficientes datos disponibles para hacer un análisis en {country}.\n\n"

# Generar las oraciones por párrafo
for direction, countries in variations.items():
    if direction == 'aumentó':
        increasing_countries = [f"{country[0]} ({country[1]:.1f}%)" for country in countries]
        analysis_text += f"En la semana de {week}, el intercambio se incrementó en {', '.join(increasing_countries)} respecto a la semana inicial. "
    elif direction == 'disminuyó':
        decreasing_countries = [f"{country[0]} ({country[1]:.1f}%)" for country in countries]
        analysis_text += f"Sin embargo, el intercambio disminuyó para {', '.join(decreasing_countries)} para el mismo período.\n"

print(analysis_text)


En la semana de 30 de June, el intercambio se incrementó en ars (108.8%), cop (67.0%), mxn (1.3%) respecto a la semana inicial. Sin embargo, el intercambio disminuyó para brl (-9.8%) para el mismo período.



In [12]:
import pandas as pd

# 1. Calcular el total de volumen para cada fecha
total_countries_grouped_df['total_volume'] = total_countries_grouped_df.groupby('created_at')['volumeUSD'].transform('sum')

# 2. Calcular la participación porcentual de cada país en el total de cada fecha
total_countries_grouped_df['relative_volume_pct'] = (total_countries_grouped_df['volumeUSD'] / total_countries_grouped_df['total_volume']) * 100

# 3. Obtener el primer y último valor de participación porcentual para cada país
first_values = total_countries_grouped_df.groupby('countries')['relative_volume_pct'].first()
last_values = total_countries_grouped_df.groupby('countries')['relative_volume_pct'].last()

# 4. Calcular el cambio en puntos porcentuales entre el último valor y el primer valor para cada país
percentage_change = last_values - first_values

# 5. Calcular la estructura relativa final
final_structure = last_values

# Crear un DataFrame con los resultados
result_df = pd.DataFrame({'First_Value': first_values, 'Last_Value': last_values, 'Percentage_Points_Change': percentage_change, 'Final_Structure': final_structure})

# Generar el párrafo de texto
countries_changes = []
for country, change, final_structure in zip(percentage_change.index, percentage_change, final_structure):
    countries_changes.append(f"{country.upper()} ha variado en {change:.1f} puntos porcentuales, alcanzando una estructura relativa final del {final_structure:.1f}%")

countries_text = ", ".join(countries_changes[:-1]) + f", y {countries_changes[-1]}."
print(f"La participación relativa en las operaciones totales de cada país ha variado para {countries_text}.")


La participación relativa en las operaciones totales de cada país ha variado para ARS ha variado en 5.2 puntos porcentuales, alcanzando una estructura relativa final del 11.0%, BRL ha variado en -1.6 puntos porcentuales, alcanzando una estructura relativa final del 7.2%, COP ha variado en 2.6 puntos porcentuales, alcanzando una estructura relativa final del 7.5%, y MXN ha variado en -6.2 puntos porcentuales, alcanzando una estructura relativa final del 74.3%..


Por cripto

In [13]:
from datetime import datetime, timedelta
import plotly.graph_objects as go

# Obtén la fecha de hoy
hoy = datetime.now().date()

# Define la fecha desde la que deseas mostrar los datos (15 de febrero)
fecha_inicio = datetime(2024, 2, 18).date()

# Define la fecha final para mostrar los datos (17 de marzo)
fecha_final = datetime(2024, 7, 29).date()

# Filtra las filas para incluir solo las desde la fecha de inicio hasta la fecha final
total_assets_selected_columns = merged_df[(merged_df['created_at'].dt.date >= fecha_inicio) & (merged_df['created_at'].dt.date <= fecha_final)]

# Selecciona solo las tres columnas sin la de fecha
total_assets_selected_columns = total_assets_selected_columns[['created_at', 'asset', 'volumeUSD']]

# Group by day and asset, sum the volumeUSD
total_assets_grouped_df = total_assets_selected_columns.groupby([pd.Grouper(key='created_at', freq='w'), 'asset']).sum().reset_index()

# Define colores fijos para cada activo
color_dict = {
    'btc': '#3A6BAC',
    'usdc': '#427F6D',
    'usdt': '#EDB041',
    'eth': '#AA322F',
    'dai': '#706F6F'
}

# Create a bar chart with plotly using daily values
total_assets_fig = go.Figure()  # Cambia el nombre de la variable a total_assets_fig

for asset in total_assets_grouped_df['asset'].unique():
    df_asset = total_assets_grouped_df[total_assets_grouped_df['asset'] == asset]
    
    # Obtén el color correspondiente al activo
    color = color_dict.get(asset, '#000000')  # Default color black if asset not found
    
    total_assets_fig.add_trace(go.Bar(
        x=df_asset['created_at'],
        y=df_asset['volumeUSD'],
        name=asset,
        marker=dict(color=color),
    ))

# Personaliza el gráfico
total_assets_fig.update_layout(
    title='<b>Cryptoasset exchange vis-a-vis local currencies by type of crypto<b>',
    title_font_color="#AA322F",    
    xaxis_title='Day',
    yaxis_title='Volume (USD)',
    barmode='stack',  # Ensure bars are stacked
    legend_title='Assets',
)

# Muestra el gráfico
total_assets_fig.show()


In [14]:
import pandas as pd

# Convertir la columna 'created_at' a tipo datetime
total_assets_grouped_df['created_at'] = pd.to_datetime(total_assets_grouped_df['created_at'])

# Ordenar el DataFrame por fecha ascendente
total_assets_grouped_df = total_assets_grouped_df.sort_values(by='created_at')

# Calcular las variaciones porcentuales entre fechas para cada activo
variations_between_dates = total_assets_grouped_df.pivot(index='created_at', columns='asset', values='volumeUSD').pct_change() * 100

# Filtrar las variaciones porcentuales para eliminar NaT
variations_between_dates = variations_between_dates.dropna()

# Calcular las variaciones de cada activo para la última fecha respecto de la inicial
initial_values = total_assets_grouped_df.groupby('asset')['volumeUSD'].first()
final_values = total_assets_grouped_df.groupby('asset')['volumeUSD'].last()
structure_percent_change = ((final_structure - initial_structure) / initial_structure) * 100

# Calcular la estructura relativa de activos para el primer y último momento
initial_structure = (initial_values / initial_values.sum()) * 100
final_structure = (final_values / final_values.sum()) * 100

# Calcular el cambio en la estructura relativa en puntos porcentuales
structure_change = final_structure - initial_structure

# Calcular la desviación estándar de las variaciones
desv_std_variaciones = variations_between_dates.std()

# Mostrar los resultados en un análisis de texto
print("\nVariaciones Porcentuales Entre Fechas:")
for date, row in variations_between_dates.iterrows():
    print(f"Fecha: {date.strftime('%Y-%m-%d')}")
    for asset, variation in row.items():
        if abs(variation) > 0.1 * desv_std_variaciones[asset]:
            print(f"- {asset}: {variation:.2f}%")

# Mostrar el cambio porcentual en la estructura relativa de activos
print("\nCambio porcentual entre el último dato y el inicial:")
for asset, percent_change in structure_percent_change.items():
    print(f"- {asset}: {percent_change:.2f}%")

print("\nEstructura Relativa de Activos:")
print("En el primer momento:")
for asset, structure in initial_structure.items():
    print(f"- {asset}: {structure:.2f}%")
print("\nEn el último momento:")
for asset, structure in final_structure.items():
    print(f"- {asset}: {structure:.2f}%")
print("\nCambio en la Estructura Relativa de Activos:")
for asset, change in structure_change.items():
    print(f"- {asset}: {change:.2f} p.p.")


Variaciones Porcentuales Entre Fechas:
Fecha: 2024-03-03
- btc: 18.07%
- dai: -42.47%
- eth: 104.55%
- usdc: 98.15%
- usdt: 17.25%
Fecha: 2024-03-10
- btc: 10.86%
- dai: 51.08%
- eth: 26.18%
- usdc: 33.51%
- usdt: 1.66%
Fecha: 2024-03-17
- btc: -14.21%
- dai: -27.56%
- eth: 19.04%
- usdc: -12.04%
- usdt: 7.44%
Fecha: 2024-03-24
- btc: -19.49%
- eth: -49.46%
- usdc: -25.83%
- usdt: -16.40%
Fecha: 2024-03-31
- btc: -23.74%
- dai: -51.50%
- usdc: -19.48%
- usdt: -8.76%
Fecha: 2024-04-07
- btc: 25.63%
- dai: 26.95%
- eth: -13.98%
- usdc: -12.22%
- usdt: 17.32%
Fecha: 2024-04-14
- btc: 16.80%
- dai: 35.79%
- eth: 45.83%
- usdc: 14.80%
- usdt: 10.43%
Fecha: 2024-04-21
- btc: 5.72%
- eth: -17.24%
- usdt: -13.43%
Fecha: 2024-04-28
- btc: -41.71%
- dai: -40.84%
- eth: -29.89%
- usdc: -28.53%
- usdt: -6.54%
Fecha: 2024-05-05
- btc: 74.56%
- dai: 38.78%
- eth: 22.07%
- usdc: 30.40%
- usdt: 17.28%
Fecha: 2024-05-12
- btc: -43.33%
- dai: -61.73%
- eth: -30.39%
- usdc: -24.62%
Fecha: 2024-05-19
- b

In [15]:
import pandas as pd

# Assuming you have a DataFrame named 'total_assets_grouped_df'

# Convertir la columna 'created_at' a tipo datetime
total_assets_grouped_df['created_at'] = pd.to_datetime(total_assets_grouped_df['created_at'])

# Ordenar el DataFrame por fecha ascendente
total_assets_grouped_df = total_assets_grouped_df.sort_values(by='created_at')

# Calcular las variaciones porcentuales entre fechas para cada activo
variations_between_dates = total_assets_grouped_df.pivot(index='created_at', columns='asset', values='volumeUSD').pct_change() * 100

# Filtrar las variaciones porcentuales para eliminar NaN
variations_between_dates = variations_between_dates.dropna()

# Calcular las variaciones de cada activo para la última fecha respecto de la inicial
initial_values = total_assets_grouped_df.groupby('asset')['volumeUSD'].first()
final_values = total_assets_grouped_df.groupby('asset')['volumeUSD'].last()
structure_percent_change = ((final_values - initial_values) / initial_values) * 100

# Excluir DAI de la estructura relativa de activos
initial_values = initial_values.drop('DAI', errors='ignore')
final_values = final_values.drop('DAI', errors='ignore')
structure_percent_change = structure_percent_change.drop('DAI', errors='ignore')

# Calcular la estructura relativa de activos para el primer y último momento
initial_structure = (initial_values / initial_values.sum()) * 100
final_structure = (final_values / final_values.sum()) * 100

# Calcular el cambio en la estructura relativa en puntos porcentuales
structure_change = final_structure - initial_structure

# Calcular la desviación estándar de las variaciones
desv_std_variaciones = variations_between_dates.std()

# Generar el análisis de texto
analysis_text = ""

# Análisis de variaciones porcentuales entre fechas
analysis_text += "\nVariaciones Porcentuales Entre Fechas:\n"
for date, row in variations_between_dates.iterrows():
    analysis_text += f"Fecha: {date.strftime('%Y-%m-%d')}\n"
    for asset, variation in row.items():
        if abs(variation) > 0.1 * desv_std_variaciones[asset]:
            analysis_text += f"- {asset}: {variation:.2f}%\n"

# Análisis de cambio porcentual en la estructura relativa de activos
analysis_text += "\nCambio porcentual entre el último dato y el inicial:\n"
for asset, percent_change in structure_percent_change.items():
    analysis_text += f"- {asset}: {percent_change:.2f}%\n"

analysis_text += "\nEstructura Relativa de Activos:\n"
analysis_text += "En el primer momento:\n"
for asset, structure in initial_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nEn el último momento:\n"
for asset, structure in final_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nCambio en la Estructura Relativa de Activos:\n"
for asset, change in structure_change.items():
    analysis_text += f"- {asset}: {change:.2f} p.p.\n"

# Imprimir el análisis de texto
print(analysis_text)



Variaciones Porcentuales Entre Fechas:
Fecha: 2024-03-03
- btc: 18.07%
- dai: -42.47%
- eth: 104.55%
- usdc: 98.15%
- usdt: 17.25%
Fecha: 2024-03-10
- btc: 10.86%
- dai: 51.08%
- eth: 26.18%
- usdc: 33.51%
- usdt: 1.66%
Fecha: 2024-03-17
- btc: -14.21%
- dai: -27.56%
- eth: 19.04%
- usdc: -12.04%
- usdt: 7.44%
Fecha: 2024-03-24
- btc: -19.49%
- eth: -49.46%
- usdc: -25.83%
- usdt: -16.40%
Fecha: 2024-03-31
- btc: -23.74%
- dai: -51.50%
- usdc: -19.48%
- usdt: -8.76%
Fecha: 2024-04-07
- btc: 25.63%
- dai: 26.95%
- eth: -13.98%
- usdc: -12.22%
- usdt: 17.32%
Fecha: 2024-04-14
- btc: 16.80%
- dai: 35.79%
- eth: 45.83%
- usdc: 14.80%
- usdt: 10.43%
Fecha: 2024-04-21
- btc: 5.72%
- eth: -17.24%
- usdt: -13.43%
Fecha: 2024-04-28
- btc: -41.71%
- dai: -40.84%
- eth: -29.89%
- usdc: -28.53%
- usdt: -6.54%
Fecha: 2024-05-05
- btc: 74.56%
- dai: 38.78%
- eth: 22.07%
- usdc: 30.40%
- usdt: 17.28%
Fecha: 2024-05-12
- btc: -43.33%
- dai: -61.73%
- eth: -30.39%
- usdc: -24.62%
Fecha: 2024-05-19
- b

## Evolución por país y criptoactivo

**Argentina**

In [16]:
import plotly.graph_objects as go
import pandas as pd

# Assume 'merged_df' now has a 'volumeUSD' column after previous operations

# Make sure 'created_at' is of datetime type
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])

# Filter data only for Argentina (via 'ars' currency in 'book')
argentina_df = merged_df[(merged_df['book'].str.contains('ars')) & (merged_df['created_at'] >= '2022-02-18')]

# Group by month and asset, summing the volume in USD
argentina_grouped_df = argentina_df.groupby([argentina_df['created_at'].dt.to_period("m"), 'book'])['volumeUSD'].sum().reset_index()

# Convert dates from Period type to simple date format
argentina_grouped_df['created_at'] = argentina_grouped_df['created_at'].dt.to_timestamp()

# Define colores fijos para cada par
color_dict = {
    'btc': '#3A6BAC',
    'usd': '#427F6D',
    'usdt': '#EDB041',
    'eth': '#AA322F',
    'dai': '#706F6F'
}

# Create stacked bar chart by asset
fig = go.Figure()

unique_books = argentina_grouped_df['book'].unique()

for i, book in enumerate(unique_books):
    book_data = argentina_grouped_df[argentina_grouped_df['book'] == book]
    
    # Obtén el color correspondiente al par
    pair_color = color_dict.get(book.split('_')[0], '#000000')  # Default color black if pair not found
    
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ))

# Customize the chart layout
fig.update_layout(
    title='<b>Evolution of crypto asset Exchange vis-a-vis ARS on Bitso, by class<b>',
    title_font_color="#AA322F",
    yaxis_title='USD',
    barmode='stack',  # Stack the bars
)

# Show the chart
fig.show()


/tmp/ipykernel_19/4173784886.py:13: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [17]:
# Asumiendo que ya tienes importado pandas y tienes tu DataFrame argentina_grouped_df

# Especifica la ruta y el nombre del archivo Excel donde deseas exportar el DataFrame
ruta_archivo_excel = '/kaggle/working/argentina_grouped_df.xlsx'

# Exporta el DataFrame a un archivo de Excel
argentina_grouped_df.to_excel(ruta_archivo_excel, index=False)

# Imprime un mensaje para confirmar que se ha exportado correctamente
print("DataFrame exportado exitosamente a:", ruta_archivo_excel)


DataFrame exportado exitosamente a: /kaggle/working/argentina_grouped_df.xlsx


In [18]:
import pandas as pd

# Suponiendo que ya tienes el DataFrame 'argentina_grouped_df' creado

# Convertir la columna 'created_at' a tipo datetime
argentina_grouped_df.index = pd.to_datetime(argentina_grouped_df.index)

# Ordenar el DataFrame por fecha ascendente
argentina_grouped_df = argentina_grouped_df.sort_index()

# Calcular las variaciones porcentuales entre fechas para cada activo
variations_between_dates = argentina_grouped_df.pivot(columns='book', values='volumeUSD').pct_change() * 100

# Filtrar las variaciones porcentuales para eliminar NaN
variations_between_dates = variations_between_dates.dropna()

# Calcular las variaciones de cada activo para la última fecha respecto de la inicial
initial_values = argentina_grouped_df.groupby('book')['volumeUSD'].first()
final_values = argentina_grouped_df.groupby('book')['volumeUSD'].last()
structure_percent_change = ((final_values - initial_values) / initial_values) * 100

# Excluir DAI de la estructura relativa de activos
initial_values = initial_values.drop('dai_ars', errors='ignore')
final_values = final_values.drop('dai_ars', errors='ignore')
structure_percent_change = structure_percent_change.drop('dai_ars', errors='ignore')

# Calcular la estructura relativa de activos para el primer y último momento
initial_structure = (initial_values / initial_values.sum()) * 100
final_structure = (final_values / final_values.sum()) * 100

# Calcular el cambio en la estructura relativa en puntos porcentuales
structure_change = final_structure - initial_structure

# Calcular la desviación estándar de las variaciones
desv_std_variaciones = variations_between_dates.std()

# Generar el análisis de texto
analysis_text = ""

# Análisis de variaciones porcentuales entre fechas
analysis_text += "\nVariaciones Porcentuales Entre Fechas:\n"
for date, row in variations_between_dates.iterrows():
    analysis_text += f"Fecha: {date.strftime('%Y-%m-%d')}\n"
    for asset, variation in row.items():
        if abs(variation) > 1 * desv_std_variaciones[asset]:
            analysis_text += f"- {asset}: {variation:.2f}%\n"

# Análisis de cambio porcentual en la estructura relativa de activos
analysis_text += "\nCambio porcentual entre el último dato y el inicial:\n"
for asset, percent_change in structure_percent_change.items():
    analysis_text += f"- {asset}: {percent_change:.2f}%\n"

analysis_text += "\nEstructura Relativa de Activos:\n"
analysis_text += "En el primer momento:\n"
for asset, structure in initial_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nEn el último momento:\n"
for asset, structure in final_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nCambio en la Estructura Relativa de Activos:\n"
for asset, change in structure_change.items():
    analysis_text += f"- {asset}: {change:.2f} p.p.\n"

# Imprimir el análisis de texto
print(analysis_text)



Variaciones Porcentuales Entre Fechas:
Fecha: 1970-01-01
- usdt_ars: 152.43%
Fecha: 1970-01-01
- btc_ars: 76.21%
Fecha: 1970-01-01
- dai_ars: 150.32%
Fecha: 1970-01-01
- eth_ars: 268.44%
Fecha: 1970-01-01
- usd_ars: 174.19%
Fecha: 1970-01-01
- usdt_ars: 115.04%
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
- btc_ars: -33.74%
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
- btc_ars: -39.82%
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01
Fecha: 1970-01-01

Cambio porcentual entre el último dato y el inicial:
- btc_ars: 428.67%
- eth_ars: -35.94%
- usd_ars: 54.01%
- usdt_ars: 80938.21%

Estructura Relativa de Activos:
En el primer momento:
- btc_ars: 2.09%
- eth_ars: 4.08%
- usd_ars: 92.96%
- usdt_ars: 0.88%

En el último momento:
- btc_ars: 1.27%
- eth_ars: 0.30%
- usd_ars: 16.47%
- usdt_

**Brasil**

In [19]:
import plotly.graph_objects as go
import pandas as pd

# Asumimos que 'merged_df' ya tiene una columna 'volumeUSD' después de las operaciones anteriores

# Asegúrate de que 'created_at' sea de tipo datetime
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])

# Filtra los datos solo para Brasil (a través de la moneda 'brl' en 'book')
brazil_df = merged_df[(merged_df['book'].str.contains('brl')) & (merged_df['created_at'] >= '2024-02-18')]

# Agrupa por semana y activo, sumando el volumen en USD
brazil_grouped_df = brazil_df.groupby([brazil_df['created_at'].dt.to_period("w"), 'book'])['volumeUSD'].sum().reset_index()

# Convierte las fechas de tipo Period a formato de fecha simple
brazil_grouped_df['created_at'] = brazil_grouped_df['created_at'].dt.to_timestamp()

# Define colores fijos para cada par
color_dict = {
    'btc': '#3A6BAC',
    'usd': '#427F6D',
    'usdt': '#EDB041',
    'eth': '#AA322F',
    'dai': '#706F6F'
}

# Create stacked bar chart by asset
fig = go.Figure()

unique_books = brazil_grouped_df['book'].unique()

for i, book in enumerate(unique_books):
    book_data = brazil_grouped_df[brazil_grouped_df['book'] == book]
    
    # Obtén el color correspondiente al par
    pair_color = color_dict.get(book.split('_')[0], '#000000')  # Default color black if pair not found
    
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ))

# Customize the chart layout
fig.update_layout(
    title='<b>Crypto Asset Exchange Vis-a-vis BRL on Bitso<b>',
    title_font_color="#AA322F",
    yaxis_title='USD',
    barmode='stack',  # Stack the bars
)

# Show the chart
fig.show()


/tmp/ipykernel_19/1181644807.py:13: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [20]:
import pandas as pd

# Suponiendo que ya tienes el DataFrame 'brazil_grouped_df' creado

# Convertir la columna 'created_at' a tipo datetime
brazil_grouped_df['created_at'] = pd.to_datetime(brazil_grouped_df['created_at'])

# Ordenar el DataFrame por fecha ascendente
brazil_grouped_df = brazil_grouped_df.sort_values(by='created_at')

# Calcular las variaciones porcentuales entre fechas para cada activo
variations_between_dates = brazil_grouped_df.pivot_table(index='created_at', columns='book', values='volumeUSD').pct_change() * 100

# Filtrar las variaciones porcentuales para eliminar NaN
variations_between_dates = variations_between_dates.dropna()

# Calcular las variaciones de cada activo para la última fecha respecto de la inicial
initial_values = brazil_grouped_df.groupby('book')['volumeUSD'].first()
final_values = brazil_grouped_df.groupby('book')['volumeUSD'].last()
structure_percent_change = ((final_values - initial_values) / initial_values) * 100

# Excluir DAI de la estructura relativa de activos
initial_values = initial_values.drop('dai_brl', errors='ignore')
final_values = final_values.drop('dai_brl', errors='ignore')
structure_percent_change = structure_percent_change.drop('dai_brl', errors='ignore')

# Calcular la estructura relativa de activos para el primer y último momento
initial_structure = (initial_values / initial_values.sum()) * 100
final_structure = (final_values / final_values.sum()) * 100

# Calcular el cambio en la estructura relativa en puntos porcentuales
structure_change = final_structure - initial_structure

# Calcular la desviación estándar de las variaciones
desv_std_variaciones = variations_between_dates.std()

# Generar el análisis de texto
analysis_text = ""

# Análisis de variaciones porcentuales entre fechas
analysis_text += "\nVariaciones Porcentuales Entre Fechas:\n"
for date, row in variations_between_dates.iterrows():
    analysis_text += f"Fecha: {date.strftime('%Y-%m-%d')}\n"
    for asset, variation in row.items():
        if abs(variation) > 0.1 * desv_std_variaciones[asset]:
            analysis_text += f"- {asset}: {variation:.2f}%\n"

# Análisis de cambio porcentual en la estructura relativa de activos
analysis_text += "\nCambio porcentual entre el último dato y el inicial:\n"
for asset, percent_change in structure_percent_change.items():
    analysis_text += f"- {asset}: {percent_change:.2f}%\n"

analysis_text += "\nEstructura Relativa de Activos:\n"
analysis_text += "En el primer momento:\n"
for asset, structure in initial_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nEn el último momento:\n"
for asset, structure in final_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nCambio en la Estructura Relativa de Activos:\n"
for asset, change in structure_change.items():
    analysis_text += f"- {asset}: {change:.2f} p.p.\n"

# Imprimir el análisis de texto
print(analysis_text)



Variaciones Porcentuales Entre Fechas:
Fecha: 2024-02-26
- btc_brl: 143.72%
- eth_brl: 78.36%
- usd_brl: 110.53%
- usdt_brl: -30.84%
Fecha: 2024-03-04
- btc_brl: 14.77%
- eth_brl: 82.76%
- usd_brl: 51.14%
- usdt_brl: -7.62%
Fecha: 2024-03-11
- btc_brl: 26.90%
- eth_brl: 39.14%
- usd_brl: 8.77%
- usdt_brl: -17.64%
Fecha: 2024-03-18
- btc_brl: -43.07%
- eth_brl: -40.59%
- usd_brl: -53.98%
- usdt_brl: -22.56%
Fecha: 2024-03-25
- btc_brl: -7.37%
- eth_brl: -61.08%
- usd_brl: -33.09%
- usdt_brl: 5.48%
Fecha: 2024-04-01
- btc_brl: 30.54%
- eth_brl: 39.57%
- usd_brl: 42.43%
- usdt_brl: 25.41%
Fecha: 2024-04-08
- btc_brl: 60.00%
- eth_brl: 187.82%
- usd_brl: 65.92%
- usdt_brl: 37.68%
Fecha: 2024-04-15
- usd_brl: -19.28%
- usdt_brl: -34.46%
Fecha: 2024-04-22
- btc_brl: -46.58%
- eth_brl: -56.05%
- usd_brl: -36.56%
Fecha: 2024-04-29
- btc_brl: 25.70%
- eth_brl: 27.53%
- usd_brl: 48.98%
- usdt_brl: 51.62%
Fecha: 2024-05-06
- btc_brl: -34.70%
- eth_brl: -36.69%
- usd_brl: 15.19%
- usdt_brl: -8.69

**Colombia**

In [21]:
import plotly.graph_objects as go
import pandas as pd

# Asumimos que 'merged_df' ya tiene una columna 'volumeUSD' después de las operaciones anteriores

# Asegúrate de que 'created_at' sea de tipo datetime
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])

# Filtra los datos solo para Colombia (a través de la moneda 'cop' en 'book')
colombia_df = merged_df[(merged_df['book'].str.contains('cop')) & (merged_df['created_at'] >= '2024-02-18')]

# Agrupa por semana y activo, sumando el volumen en USD
colombia_grouped_df = colombia_df.groupby([colombia_df['created_at'].dt.to_period("w"), 'book'])['volumeUSD'].sum().reset_index()

# Convierte las fechas de tipo Period a formato de fecha simple
colombia_grouped_df['created_at'] = colombia_grouped_df['created_at'].dt.to_timestamp()

# Define colores fijos para cada par
color_dict = {
    'btc': '#3A6BAC',
    'usd': '#427F6D',
    'usdt': '#EDB041',
    'eth': '#AA322F',
    'dai': '#706F6F'
}

# Create stacked bar chart by asset
fig = go.Figure()

unique_books = colombia_grouped_df['book'].unique()

for i, book in enumerate(unique_books):
    book_data = colombia_grouped_df[colombia_grouped_df['book'] == book]
    
    # Obtén el color correspondiente al par
    pair_color = color_dict.get(book.split('_')[0], '#000000')  # Default color black if pair not found
    
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ))

# Customize the chart layout
fig.update_layout(
    title='<b>Crypto Asset Exchange Vis-a-vis COP on Bitso<b>',
    title_font_color="#AA322F",
    yaxis_title='USD',
    barmode='stack',  # Stack the bars
)

# Show the chart
fig.show()


/tmp/ipykernel_19/3078034873.py:13: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [22]:
import pandas as pd

# Suponiendo que ya tienes el DataFrame 'colombia_grouped_df' creado

# Convertir la columna 'created_at' a tipo datetime
colombia_grouped_df['created_at'] = pd.to_datetime(colombia_grouped_df['created_at'])

# Ordenar el DataFrame por fecha ascendente
colombia_grouped_df = colombia_grouped_df.sort_values(by='created_at')

# Calcular las variaciones porcentuales entre fechas para cada activo
variations_between_dates = colombia_grouped_df.pivot_table(index='created_at', columns='book', values='volumeUSD').pct_change() * 100

# Filtrar las variaciones porcentuales para eliminar NaN
variations_between_dates = variations_between_dates.dropna()

# Calcular las variaciones de cada activo para la última fecha respecto de la inicial
initial_values = colombia_grouped_df.groupby('book')['volumeUSD'].first()
final_values = colombia_grouped_df.groupby('book')['volumeUSD'].last()
structure_percent_change = ((final_values - initial_values) / initial_values) * 100

# Excluir DAI de la estructura relativa de activos
initial_values = initial_values.drop('dai_col', errors='ignore')
final_values = final_values.drop('dai_col', errors='ignore')
structure_percent_change = structure_percent_change.drop('dai_col', errors='ignore')

# Calcular la estructura relativa de activos para el primer y último momento
initial_structure = (initial_values / initial_values.sum()) * 100
final_structure = (final_values / final_values.sum()) * 100

# Calcular el cambio en la estructura relativa en puntos porcentuales
structure_change = final_structure - initial_structure

# Calcular la desviación estándar de las variaciones
desv_std_variaciones = variations_between_dates.std()

# Generar el análisis de texto
analysis_text = ""

# Análisis de variaciones porcentuales entre fechas
analysis_text += "\nVariaciones Porcentuales Entre Fechas:\n"
for date, row in variations_between_dates.iterrows():
    analysis_text += f"Fecha: {date.strftime('%Y-%m-%d')}\n"
    for asset, variation in row.items():
        if abs(variation) > 1 * desv_std_variaciones[asset]:
            analysis_text += f"- {asset}: {variation:.2f}%\n"

# Análisis de cambio porcentual en la estructura relativa de activos
analysis_text += "\nCambio porcentual entre el último dato y el inicial:\n"
for asset, percent_change in structure_percent_change.items():
    analysis_text += f"- {asset}: {percent_change:.2f}%\n"

analysis_text += "\nEstructura Relativa de Activos:\n"
analysis_text += "En el primer momento:\n"
for asset, structure in initial_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nEn el último momento:\n"
for asset, structure in final_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nCambio en la Estructura Relativa de Activos:\n"
for asset, change in structure_change.items():
    analysis_text += f"- {asset}: {change:.2f} p.p.\n"

# Imprimir el análisis de texto
print(analysis_text)



Variaciones Porcentuales Entre Fechas:
Fecha: 2024-02-26
- btc_cop: 280.35%
Fecha: 2024-03-04
Fecha: 2024-03-11
Fecha: 2024-03-18
Fecha: 2024-03-25
Fecha: 2024-04-01
- eth_cop: 238.93%
Fecha: 2024-04-08
- eth_cop: 205.15%
Fecha: 2024-04-15
- usd_cop: 194.46%
Fecha: 2024-04-22
- usdt_cop: 162.99%
Fecha: 2024-04-29
Fecha: 2024-05-06
- usd_cop: 550.45%
Fecha: 2024-05-13
Fecha: 2024-05-20
- eth_cop: 178.95%
Fecha: 2024-05-27
- btc_cop: -87.42%
Fecha: 2024-06-03
- btc_cop: 194.60%
- eth_cop: 700.62%
- usdt_cop: 248.98%
Fecha: 2024-06-10
- usd_cop: 326.87%
Fecha: 2024-06-17
Fecha: 2024-06-24
- btc_cop: 97.00%
Fecha: 2024-07-01
Fecha: 2024-07-08
- usd_cop: 226.11%
Fecha: 2024-07-15
Fecha: 2024-07-22

Cambio porcentual entre el último dato y el inicial:
- btc_cop: 5.83%
- eth_cop: 68.74%
- usd_cop: -62.83%
- usdt_cop: 76.95%

Estructura Relativa de Activos:
En el primer momento:
- btc_cop: 0.65%
- eth_cop: 0.64%
- usd_cop: 6.77%
- usdt_cop: 91.95%

En el último momento:
- btc_cop: 0.41%
- eth

**México**

In [23]:
import plotly.graph_objects as go
import pandas as pd

# Asumimos que 'merged_df' ya tiene una columna 'volumeUSD' después de las operaciones anteriores

# Asegúrate de que 'created_at' sea de tipo datetime
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])

# Filtra los datos solo para México (a través de la moneda 'mxn' en 'book')
mexico_df = merged_df[(merged_df['book'].str.contains('mxn')) & (merged_df['created_at'] >= '2024-02-18')]

# Agrupa por semana y activo, sumando el volumen en USD
mexico_grouped_df = mexico_df.groupby([mexico_df['created_at'].dt.to_period("w"), 'book'])['volumeUSD'].sum().reset_index()

# Convierte las fechas de tipo Period a formato de fecha simple
mexico_grouped_df['created_at'] = mexico_grouped_df['created_at'].dt.to_timestamp()

# Define colores fijos para cada par
color_dict = {
    'btc': '#3A6BAC',
    'usd': '#427F6D',
    'usdt': '#EDB041',
    'eth': '#AA322F',
    'dai': '#706F6F'
}

# Create stacked bar chart by asset
fig = go.Figure()

unique_books = mexico_grouped_df['book'].unique()

for i, book in enumerate(unique_books):
    book_data = mexico_grouped_df[mexico_grouped_df['book'] == book]
    
    # Obtén el color correspondiente al par
    pair_color = color_dict.get(book.split('_')[0], '#000000')  # Default color black if pair not found
    
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ))

# Customize the chart layout
fig.update_layout(
    title='<b>Crypto Asset Exchange Vis-a-vis MXN on Bitso<b>',
    title_font_color="#AA322F",
    yaxis_title='USD',
    barmode='stack',  # Stack the bars
)

# Show the chart
fig.show()

/tmp/ipykernel_19/1285519441.py:13: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [24]:
import pandas as pd

# Suponiendo que ya tienes el DataFrame 'mexico_grouped_df' creado

# Convertir la columna 'created_at' a tipo datetime
mexico_grouped_df['created_at'] = pd.to_datetime(mexico_grouped_df['created_at'])

# Ordenar el DataFrame por fecha ascendente
mexico_grouped_df = mexico_grouped_df.sort_values(by='created_at')

# Calcular las variaciones porcentuales entre fechas para cada activo
variations_between_dates = mexico_grouped_df.pivot_table(index='created_at', columns='book', values='volumeUSD').pct_change() * 100

# Filtrar las variaciones porcentuales para eliminar NaN
variations_between_dates = variations_between_dates.dropna()

# Calcular las variaciones de cada activo para la última fecha respecto de la inicial
initial_values = mexico_grouped_df.groupby('book')['volumeUSD'].first()
final_values = mexico_grouped_df.groupby('book')['volumeUSD'].last()
structure_percent_change = ((final_values - initial_values) / initial_values) * 100

# Excluir DAI de la estructura relativa de activos
initial_values = initial_values.drop('dai_mex', errors='ignore')
final_values = final_values.drop('dai_mex', errors='ignore')
structure_percent_change = structure_percent_change.drop('dai_mex', errors='ignore')

# Calcular la estructura relativa de activos para el primer y último momento
initial_structure = (initial_values / initial_values.sum()) * 100
final_structure = (final_values / final_values.sum()) * 100

# Calcular el cambio en la estructura relativa en puntos porcentuales
structure_change = final_structure - initial_structure

# Calcular la desviación estándar de las variaciones
desv_std_variaciones = variations_between_dates.std()

# Generar el análisis de texto
analysis_text = ""

# Análisis de variaciones porcentuales entre fechas
analysis_text += "\nVariaciones Porcentuales Entre Fechas:\n"
for date, row in variations_between_dates.iterrows():
    analysis_text += f"Fecha: {date.strftime('%Y-%m-%d')}\n"
    for asset, variation in row.items():
        if abs(variation) > 1 * desv_std_variaciones[asset]:
            analysis_text += f"- {asset}: {variation:.2f}%\n"

# Análisis de cambio porcentual en la estructura relativa de activos
analysis_text += "\nCambio porcentual entre el último dato y el inicial:\n"
for asset, percent_change in structure_percent_change.items():
    analysis_text += f"- {asset}: {percent_change:.2f}%\n"

analysis_text += "\nEstructura Relativa de Activos:\n"
analysis_text += "En el primer momento:\n"
for asset, structure in initial_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nEn el último momento:\n"
for asset, structure in final_structure.items():
    analysis_text += f"- {asset}: {structure:.2f}%\n"
analysis_text += "\nCambio en la Estructura Relativa de Activos:\n"
for asset, change in structure_change.items():
    analysis_text += f"- {asset}: {change:.2f} p.p.\n"

# Imprimir el análisis de texto
print(analysis_text)



Variaciones Porcentuales Entre Fechas:
Fecha: 2024-02-26
- eth_mxn: 114.19%
- usd_mxn: 103.02%
- usdt_mxn: 24.02%
Fecha: 2024-03-04
Fecha: 2024-03-11
Fecha: 2024-03-18
- eth_mxn: -51.22%
- usdt_mxn: -24.95%
Fecha: 2024-03-25
Fecha: 2024-04-01
- usdt_mxn: 27.55%
Fecha: 2024-04-08
Fecha: 2024-04-15
Fecha: 2024-04-22
- btc_mxn: -41.21%
Fecha: 2024-04-29
- btc_mxn: 81.52%
- usd_mxn: 41.19%
Fecha: 2024-05-06
- btc_mxn: -44.21%
- usd_mxn: -42.60%
Fecha: 2024-05-13
- dai_mxn: 343.61%
- usd_mxn: 109.71%
Fecha: 2024-05-20
- eth_mxn: 141.77%
- usdt_mxn: 20.25%
Fecha: 2024-05-27
- dai_mxn: 143.51%
- usdt_mxn: -23.72%
Fecha: 2024-06-03
- btc_mxn: 72.92%
- eth_mxn: 59.28%
- usdt_mxn: 33.99%
Fecha: 2024-06-10
Fecha: 2024-06-17
Fecha: 2024-06-24
- dai_mxn: 115.42%
Fecha: 2024-07-01
Fecha: 2024-07-08
Fecha: 2024-07-15
- eth_mxn: 55.42%
Fecha: 2024-07-22
- dai_mxn: 200.44%

Cambio porcentual entre el último dato y el inicial:
- btc_mxn: -47.71%
- dai_mxn: -46.15%
- eth_mxn: 86.26%
- usd_mxn: -21.10%
-

**Todos los gráficos en un por cuadrantes**

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Supongamos que 'merged_df' tiene una columna 'volumeUSD' después de operaciones previas

# Asegúrate de que 'created_at' sea del tipo datetime
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])

# Filtrar datos para cada país y rango de fechas
start_date = '2024-02-18'
end_date = '2024-03-25'

argentina_df = merged_df[(merged_df['created_at'] >= start_date) & (merged_df['created_at'] <= end_date) & (merged_df['book'].str.contains('ars'))]
brasil_df = merged_df[(merged_df['created_at'] >= start_date) & (merged_df['created_at'] <= end_date) & (merged_df['book'].str.contains('brl'))]
colombia_df = merged_df[(merged_df['created_at'] >= start_date) & (merged_df['created_at'] <= end_date) & (merged_df['book'].str.contains('cop'))]
mexico_df = merged_df[(merged_df['created_at'] >= start_date) & (merged_df['created_at'] <= end_date) & (merged_df['book'].str.contains('mxn'))]

# Agrupar por semana y activo, sumando el volumen en USD
argentina_grouped_df = argentina_df.groupby([argentina_df['created_at'].dt.to_period("w"), 'book'])['volumeUSD'].sum().reset_index()
brasil_grouped_df = brasil_df.groupby([brasil_df['created_at'].dt.to_period("w"), 'book'])['volumeUSD'].sum().reset_index()
colombia_grouped_df = colombia_df.groupby([colombia_df['created_at'].dt.to_period("w"), 'book'])['volumeUSD'].sum().reset_index()
mexico_grouped_df = mexico_df.groupby([mexico_df['created_at'].dt.to_period("w"), 'book'])['volumeUSD'].sum().reset_index()

# Convertir fechas de tipo Period a formato de fecha simple
argentina_grouped_df['created_at'] = argentina_grouped_df['created_at'].dt.to_timestamp()
brasil_grouped_df['created_at'] = brasil_grouped_df['created_at'].dt.to_timestamp()
colombia_grouped_df['created_at'] = colombia_grouped_df['created_at'].dt.to_timestamp()
mexico_grouped_df['created_at'] = mexico_grouped_df['created_at'].dt.to_timestamp()

# Definir colores fijos para cada par
color_dict = {
    'btc': '#3A6BAC',
    'usd': '#427F6D',
    'usdt': '#EDB041',
    'eth': '#AA322F',
    'dai': '#706F6F'
}

# Crear gráfico de barras apiladas por activo
fig = make_subplots(rows=2, cols=2, subplot_titles=("Argentina", "Brasil", "Colombia", "México"))

# Crear gráfico para Argentina
for i, book in enumerate(argentina_grouped_df['book'].unique()):
    book_data = argentina_grouped_df[argentina_grouped_df['book'] == book]
    pair_color = color_dict.get(book.split('_')[0], '#000000')  # Color predeterminado negro si no se encuentra el par
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ), row=1, col=1)

# Crear gráfico para Brasil
for i, book in enumerate(brasil_grouped_df['book'].unique()):
    book_data = brasil_grouped_df[brasil_grouped_df['book'] == book]
    pair_color = color_dict.get(book.split('_')[0], '#000000')
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ), row=1, col=2)

# Crear gráfico para Colombia
for i, book in enumerate(colombia_grouped_df['book'].unique()):
    book_data = colombia_grouped_df[colombia_grouped_df['book'] == book]
    pair_color = color_dict.get(book.split('_')[0], '#000000')
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ), row=2, col=1)

# Crear gráfico para México
for i, book in enumerate(mexico_grouped_df['book'].unique()):
    book_data = mexico_grouped_df[mexico_grouped_df['book'] == book]
    pair_color = color_dict.get(book.split('_')[0], '#000000')
    fig.add_trace(go.Bar(
        x=book_data['created_at'],
        y=book_data['volumeUSD'],
        name=book,
        marker=dict(color=pair_color),
    ), row=2, col=2)

# Personalizar diseño del gráfico
fig.update_layout(
    title='<b>Cryptoasset exchange vis-a-vis local currency by countries and crypto<b>',
    title_font_color="#AA322F",
    yaxis_title='USD',
    barmode='stack'  # Barras apiladas
)

# Mostrar el gráfico
fig.show()


/tmp/ipykernel_19/3818934602.py:20: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/tmp/ipykernel_19/3818934602.py:21: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/tmp/ipykernel_19/3818934602.py:22: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/tmp/ipykernel_19/3818934602.py:23: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



## Relative comparisson by countries

In [26]:
from datetime import datetime, timedelta

# Obtén la fecha de hoy
hoy = datetime.now().date()

# Calcula la fecha de ayer
ayer = hoy - timedelta(days=10)

# Selecciona las cuatro columnas deseadas
selected_columns = merged_df[['created_at', 'countries', 'asset', 'volumeUSD']]

# Filtra las filas para incluir solo las de ayer y hoy
# selected_columns = selected_columns[(merged_df['created_at'].dt.date == ayer) | (merged_df['created_at'].dt.date == hoy)]

# Selecciona solo las tres columnas sin la de fecha
selected_columns = selected_columns[['countries', 'asset', 'volumeUSD']]

# Group by asset and sum the volumeUSD
grouped_df = selected_columns.groupby(['countries', 'asset']).sum().reset_index()

# Normalize values by country
grouped_df['normalized_volume'] = grouped_df.groupby('countries')['volumeUSD'].transform(lambda x: x / x.sum())

# Define colores fijos para cada par
color_dict = {
    'btc': '#3A6BAC',
    'usdc': '#427F6D',
    'usdt': '#EDB041',
    'eth': '#AA322F',
    'dai': '#706F6F'
}

# Create a stacked bar chart with plotly using normalized values
fig2 = go.Figure()

for asset in grouped_df['asset'].unique():
    df_asset = grouped_df[grouped_df['asset'] == asset]
    
    # Obtén el color correspondiente al activo
    color = color_dict.get(asset, '#000000')  # Default color black if asset not found
    
    fig2.add_trace(go.Bar(
        x=df_asset['countries'],
        y=df_asset['normalized_volume'],
        name=asset,
        marker=dict(color=color),
    ))

# Customize the chart
fig2.update_layout(
    title='<b>Relative Participation by Country and Asset<b>',
    title_font_color="#AA322F",    
    xaxis_title='Countries',
    yaxis_title='Relative Participation',
    barmode='stack',
    legend_title='Assets'
)

# Show the chart
fig2.show()


## Análisis estadísticos

Correlación

In [27]:
# Calcular la matriz de correlación
correlation_matrix = total_assets_grouped_df.pivot_table(index='created_at', columns='asset', values='volumeUSD').corr()

# Imprimir la matriz de correlación
print(correlation_matrix)

asset      btc       dai       eth       usdc      usdt
asset                                                  
btc    1.000000  0.589077  0.482503  0.749309  0.146880
dai    0.589077  1.000000  0.288881  0.523166 -0.282362
eth    0.482503  0.288881  1.000000  0.693253  0.300395
usdc   0.749309  0.523166  0.693253  1.000000 -0.058627
usdt   0.146880 -0.282362  0.300395 -0.058627  1.000000


In [28]:
import pandas as pd
import plotly.graph_objects as go

# Filtrar el DataFrame para excluir el activo "DAI"
total_assets_grouped_df_filtered = total_assets_grouped_df[total_assets_grouped_df['asset'] != 'dai']

# Calcular la matriz de correlación
correlation_matrix = total_assets_grouped_df_filtered.pivot_table(index='created_at', columns='asset', values='volumeUSD').corr()

# Obtener nombres de las columnas (activos)
assets = correlation_matrix.columns.tolist()

# Crear el mapa de calor utilizando Plotly
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=assets,
    y=assets,
    colorscale='RdBu'))

# Agregar valores numéricos en cada celda del mapa de calor
annotations = []
for i, asset1 in enumerate(assets):
    for j, asset2 in enumerate(assets):
        annotations.append(
            dict(
                x=asset2,
                y=asset1,
                text=str(round(correlation_matrix.values[i][j], 2)),
                showarrow=False
            )
        )

fig.update_layout(
    title='Matriz de correlación entre activos (sin DAI)',
    xaxis_title='Activos',
    yaxis_title='Activos',
    annotations=annotations,  # Agregar valores numéricos
)

# Mostrar el gráfico
fig.show()


In [29]:
# Calcular la matriz de correlación
correlation_matrix_countries = total_countries_grouped_df.pivot_table(index='created_at', columns='countries', values='volumeUSD').corr()

# Imprimir la matriz de correlación
print(correlation_matrix_countries)

countries       ars       brl       cop       mxn
countries                                        
ars        1.000000  0.061437 -0.220829  0.251052
brl        0.061437  1.000000  0.196041  0.594619
cop       -0.220829  0.196041  1.000000 -0.123117
mxn        0.251052  0.594619 -0.123117  1.000000


In [30]:
import pandas as pd
import plotly.graph_objects as go

# Calcular la matriz de correlación
correlation_matrix_countries = total_countries_grouped_df.pivot_table(index='created_at', columns='countries', values='volumeUSD').corr()

# Obtener nombres de las columnas (países)
countries = correlation_matrix_countries.columns.tolist()

# Crear el mapa de calor utilizando Plotly
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix_countries.values,
    x=countries,
    y=countries,
    colorscale='RdBu'))

# Agregar valores numéricos en cada celda del mapa de calor
annotations = []
for i, country1 in enumerate(countries):
    for j, country2 in enumerate(countries):
        annotations.append(
            dict(
                x=country2,
                y=country1,
                text=str(round(correlation_matrix_countries.values[i][j], 2)),
                showarrow=False
            )
        )

fig.update_layout(
    title='Matriz de correlación entre países',
    xaxis_title='Países',
    yaxis_title='Países',
    annotations=annotations,  # Agregar valores numéricos
)

# Mostrar el gráfico
fig.show()


In [31]:
import pandas as pd
import plotly.graph_objects as go

# Filter the DataFrame to exclude the "DAI" asset
total_assets_grouped_df_filtered = total_assets_grouped_df[total_assets_grouped_df['asset'] != 'dai']

# Calculate the correlation matrix for assets
correlation_matrix_assets = total_assets_grouped_df_filtered.pivot_table(index='created_at', columns='asset', values='volumeUSD').corr()

# Get the column names of assets
assets = correlation_matrix_assets.columns.tolist()

# Calculate the correlation matrix for countries
correlation_matrix_countries = total_countries_grouped_df.pivot_table(index='created_at', columns='countries', values='volumeUSD').corr()

# Get the column names of countries
countries = correlation_matrix_countries.columns.tolist()

# Create the combined correlation matrix
combined_correlation_matrix = pd.concat([correlation_matrix_assets, correlation_matrix_countries], keys=['Assets', 'Countries'])

# Create the heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=combined_correlation_matrix.values,
    x=combined_correlation_matrix.columns,
    y=combined_correlation_matrix.index.get_level_values(1),
    colorscale='RdBu',
    colorbar=dict(title='Correlation'),
    zmin=-1,
    zmax=1,
    zmid=0
))

# Update layout and add titles
fig.update_layout(
    title='Correlation Matrix of Assets and Countries',
    xaxis_title='Assets / Countries',
    yaxis_title='Assets / Countries',
    coloraxis_colorbar=dict(
        ticks='outside'
    )
)

# Show the plot
fig.show()


In [32]:
import plotly.graph_objects as go
from scipy.stats import f_oneway

# Split the data into two groups: btc+eth and usdc+usdt
group1 = total_assets_grouped_df[total_assets_grouped_df['asset'].isin(['btc', 'eth'])]['volumeUSD']
group2 = total_assets_grouped_df[total_assets_grouped_df['asset'].isin(['usdc', 'usdt'])]['volumeUSD']

# Perform ANOVA test
statistic, p_value = f_oneway(group1, group2)

# Create a Plotly figure for the box plot
fig = go.Figure()

# Add boxes for each group
fig.add_trace(go.Box(y=group1, name='BTC+ETH'))
fig.add_trace(go.Box(y=group2, name='USDC+USDT'))

# Add a line for the critical value of the F statistic
fig.add_shape(type="line", x0=0, y0=statistic, x1=1, y1=statistic,
              line=dict(color="red", width=2, dash="dash"))

# Annotate the p-value and conclusion of the test
if p_value < 0.05:
    conclusion_text = 'Significant Difference'
    conclusion_color = 'red'
else:
    conclusion_text = 'No Significant Difference'
    conclusion_color = 'green'

fig.add_annotation(x=1.2, y=statistic + 5,
                   text=f'p-value: {p_value:.4f}',
                   showarrow=False,
                   font=dict(size=12))
fig.add_annotation(x=1.2, y=statistic - 10,
                   text=conclusion_text,
                   showarrow=False,
                   font=dict(size=12, color=conclusion_color))

# Update the layout of the plot
fig.update_layout(title='Comparison of Transaction Volumes Between Groups',
                  xaxis_title='Asset Groups',
                  yaxis_title='Transaction Volume',
                  boxmode='group')

# Show the plot
fig.show()


In [33]:
import numpy as np

# Separar los dos grupos
btc_eth_group = total_assets_grouped_df[(total_assets_grouped_df['asset'] == 'btc') | (total_assets_grouped_df['asset'] == 'eth')]['volumeUSD']
usdc_usdt_group = total_assets_grouped_df[(total_assets_grouped_df['asset'] == 'usdc') | (total_assets_grouped_df['asset'] == 'usdt')]['volumeUSD']

# Calcular las medias de los dos grupos
mean_btc_eth = btc_eth_group.mean()
mean_usdc_usdt = usdc_usdt_group.mean()

# Calcular las desviaciones estándar de los dos grupos
std_btc_eth = btc_eth_group.std()
std_usdc_usdt = usdc_usdt_group.std()

# Calcular la desviación estándar combinada
pooled_std = np.sqrt((std_btc_eth**2 + std_usdc_usdt**2) / 2)

# Calcular Cohen's d
effect_size = (mean_btc_eth - mean_usdc_usdt) / pooled_std

print("Tamaño del efecto (Cohen's d):", effect_size)


Tamaño del efecto (Cohen's d): -1.7648709238449167


In [34]:
# Nombre del archivo Excel de salida
excel_filename = 'la_grouped_df.xlsx'

# Exportar el DataFrame a un archivo Excel
grouped_df.to_excel(excel_filename, index=False)

# Imprimir mensaje de confirmación
print(f"Los datos se han exportado exitosamente a {excel_filename}")

Los datos se han exportado exitosamente a la_grouped_df.xlsx


# Sending to Monitor

In [35]:
pip install chart_studio

  Obtaining dependency information for chart_studio from https://files.pythonhosted.org/packages/ca/ce/330794a6b6ca4b9182c38fc69dd2a9cbff60fd49421cb8648ee5fee352dc/chart_studio-1.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [36]:
import chart_studio
import chart_studio.plotly as py

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_3 = user_secrets.get_secret("CHARTSTUDIO_BCRA_API_KEY")

In [37]:
# Graf de Vol 
chart_studio.tools.set_credentials_file(username='crypto-financial-stability-monitor', 
                                        api_key=secret_value_3)
py.plot(fig, filename = 'vol-bitso-ars', auto_open=True)

'https://plotly.com/~crypto-financial-stability-monitor/2785/'

In [38]:
# Graf de Vol 
chart_studio.tools.set_credentials_file(username='crypto-financial-stability-monitor', 
                                        api_key=secret_value_3)
py.plot(fig2, filename = 'vol-bitso-al', auto_open=True)

'https://plotly.com/~crypto-financial-stability-monitor/2787/'